In [69]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import os
import sys
from dotenv import find_dotenv, load_dotenv
from sklearn.model_selection import train_test_split

# Insert project folder into Python System
load_dotenv(find_dotenv())
sys.path.append(os.getenv("PROJECT_FOLDER"))

In [70]:
""" LOAD DATA """
df = pd.read_csv("../data/processed/sample-clean-data.csv")
df = df.dropna(subset=["preprocessed_review_text"])
display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 36868 entries, 0 to 36927
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   date                      36868 non-null  object
 1   category                  36868 non-null  object
 2   sentiment                 36868 non-null  object
 3   preprocessed_review_text  36868 non-null  object
 4   id                        36868 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


None

,date,category,sentiment,preprocessed_review_text,id
0,1998-07-09,Digital_Music_5,positive,all-time favorite jazz recording 10 star best ...,0
1,1998-08-21,Digital_Music_5,negative,expected double cd known tribute day three pen...,1
2,2000-04-25,Digital_Music_5,positive,pretty good dug feel good oakland stroke vanes...,2
3,2000-04-28,Digital_Music_5,negative,people impressed must heard new beginning last...,3
4,2000-05-09,Digital_Music_5,positive,second album jackson browne hit creative strid...,4


In [78]:
""" Split Data For Training and Testing """
X_columns =["id", "date",	"category", "preprocessed_review_text"]
y_columns =["sentiment"]
df['sentiment'] = df['sentiment'].map({"positive": 1, "negative": 0}) 
X_train, X_test, y_train, y_test = train_test_split(df[X_columns], df[y_columns], stratify=df[y_columns], test_size=0.2, shuffle=True, random_state=42)

Simple Baseline Model using Logistic Regression
1. Preprocessed Text
2. Create Count Words Positive and Negative
3. Calculate the Laplacian
4. Create & feed it as a feature into logistic regression

In [100]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Generating Feature using CountVectorizer
bow_converter = CountVectorizer()
X_train_bow = bow_converter.fit_transform(X_train['preprocessed_review_text'])
X_test_bow = bow_converter.transform(X_test['preprocessed_review_text'])

# Logistic Regression
model = LogisticRegression()
model.fit(X_train_bow, y_train)
y_pred = model.predict_log_proba(X_test_bow)
model.score(X_test_bow, y_test)

c:\Users\USER\anaconda3\envs\nlp-tools\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\USER\anaconda3\envs\nlp-tools\lib\site-packages\sklearn\linear_model\_logistic.py:1407: RuntimeWarning: divide by zero encountered in log
  return np.log(self.predict_proba(X))


0.7998372660699756